## RSA - The Project

Name:  Alex Melnick

Moodle Email: alme9138@colorado.edu

*Name your file "cuidentikey".ipynb *

<hr />

# Table of Contents

### 1. Introduction  (5 points)

### 2. RSA Code Package    (10 points)
###### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 2.1 Basic tool set
###### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 2.2 First tool set
###### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 2.3 Second tool set

### 3. RSA More Code  (10 points)

###### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;  Encode
###### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;  Decode

### 4. Show a demo of encoding and decoding a message that highlights how your code works and the steps involved.   (20points)

### 5. Project Narrative AND results of exchanging codes with others.  (20 points)

### 6. Testing and comparing FME to mod.       (10 points)

### Points for 7 - 10 only available if the student is exchanging message by the Friday (midnight) before the project is due.

### 7. Breaking codes without a private key    (10 points)

### 8. Improving the brute force method of factoring      (5 points)
### 9. Examples of code breaking using improved methods.  (5 points)

### 10. Advanced options (5 points) Needed to get 100%.

### 1. Introduction to your RSA Package and Project.  Give an overview of your project and what you have learned.

This notebook contains a program that implements the RSA encryption algorithm to encrypt and decrypt messages and ciphers. RSA is a technique that takes two public keys, n and e, and encrypts a message M by performing M^e mod n. A private key, d, can be used to reverse the process to yield the message, M, from the cipher, C (M = C^d mod n). The reason this is effective is that you can publish the public keys, allowing other people to send you encrypted messages. Nobody will be able to determine what d is unless they can determine the prime factors of n. With a large enough n, this becomes practically impossible with the computational resources available today.

This project provides the means to generate public and private keys, encrypt and decrypt messages, and factor simple public keys to reverse-engineer the private key.

A guide to the interconnected functions follows:

#### Encoding a message requires:

* Converting the message (a string) into a list of integers - Convert_Text

* Generating a public key n from two prime numbers - Initialize_ned or user-provided

* Generating a public key e for public key n - Find_Public_Key_e

* Determining that e and (p-1)(q-1) are pairwise with the Euclidean Algorithm- Euclidean_Alg

* Reducing terms modulo efficiently with Fast Modular Exponentiation - FME

* Converting integers to a binary string in order to perform FME - Convert_Binary_String

#### Decoding a message requires:

* Generating a private key d which is a modular inverse of e and the bezout coefficient of e mod (p-1)(q-1)- Find_Private_Key_d and Bezout_co

* Decoding the cipher (a list of integers) - Decode

* Converting the decoded message from a list of integers to a string - Convert_Num

#### Breaking a public key requires the ability to factor n:

* Slowly, by brute force - Factorize_Basic

* Quickly, after preparing a list of prime numbers - Factorize_Improved and Generate_Prime_List

* Extremely quickly, using Pollard's Rho algorithm which leverages the Birthday "paradox" - Pollard_Rho

### 2.1
#### Basic tool set

These are functions that you'll need to pre-process the messages before the messages are encoded and decoded by the RSA algorithm. That is the reason we will be defining them first.



In [1]:
'''
This function converts a simple string and converts it to a list of integers (ascii) for each letter.
@param: text (string), a string.
@output: integer_list [list], a list of integers corresponding to the letters in text.
'''
def Convert_Text(text):
    integer_list = []
    for char in text: #Loop through each character in input string
        integer_list.append(ord(char)) #convert each character to integer (ascii) and store in integer_list
    return integer_list

print(Convert_Text("h"))

[104]


In [2]:
'''
Converts a list of integers (ascii) back into a string.
@param: integer_list [list], a list of integers corresponding to ascii characters
@output: text (string), the converted integer_list as a string
'''

def Convert_Num(integer_list):
    text = ''
    for i in integer_list: #loop through each element of integer_list
        text += chr(i) #convert the element to a character and concatenate with previously-converted characters in a string
    return text

In [3]:
'''
Converts a decimal integer into the equivalent binary number as a string
@param: decint (int), the integer to be converted
@output: bits (string), the binary expansion of decint as a string
'''

def Convert_Binary_String(decint):
    bits = 0 #collection of binary digits
    i = 0 #increments each loop to control digit for input
    while (decint > 0): #loop until all value in decint is added to bits
        bits = bits + ((decint % 2) * 10**i) #find binary value of significant digit and add to appropriate place in bits
        i += 1 #move to next digit in bits (more significant)
        decint = decint // 2 #remove "used" value from decint
    return str(bits)

Now that you're done with the basic toolset we'll move on to the first tool set which is actually involved in the RSA system.

### 2.2 
#### First tool set.



In [4]:
'''
Performs Fast Modular Exponentiation on a decimal integer.
@param: b (int), base of the term to be reduced modulo
@param: n (int), exponent of the term to be reduced modulo
@param: m (int), the modulus
@output: result, b^n mod m via FME
'''

def FME(b, n, m):
    n = Convert_Binary_String(n) #convert n to binary string
    result = 1 #will store result of iterated modulus operations
    square = b % m #initial value of b^2^0 mod m
    for i in range(len(n), 0, -1): #iterate backwards through the string
        if int(n[i-1]) == 1: #test current digit. If 1, then multiply by current square value
            result = (result * square) % m
        square = (square * square) % m #prepare square for next iteration, regardless of conditional outcome
    return result

In [5]:
'''
Calculates the greatest common divisor of a and b.
@param: a (int), one of two terms to determine the greatest common divisor of
@param: b (int), two of two terms to determine the greatest common divisor of
@output: first (int), the greatest common divisor between a and b
'''
def Euclidean_Alg(a, b):
    first = max(a, b) #place a and b in the proper order
    last = min(a, b) 
    while (last != 0): #loop until one of the two terms is reduced to zero
        hold = last #keep the second term aside so it can become the first
        last = FME(first, 1, last) #perform first mod last using the FME function and make it the now-lesser second term
        first = hold #make the now-greater second term into the first
    return first

In [6]:
'''
Calculates the bezout coefficients of two integers
@param: b (int), one of two terms to determine the bezout coefficients of
@param: a (int), the second of two terms to determine the bezout coefficients of
@output: bezoutco [int, int], a list of the two bezout coefficients
'''
def bezout_co(b, a):
    phi = a #hold modulus for later adjustment if negative
    s1t1 = [1,0] #set up two-element lists to hold coefficients
    s2t2 = [0,1]
    while (a != 0): #loop until one of the two terms is reduced to zero
        hold = a #keep the second term aside so it can become the first
        q = b // a #perform integer division on the two terms to get the quotient
        a = FME(b, 1, a) #perform first mod last using the FME function and assign it to the last term for the next iteration
        b = hold #make the now-greater second term into the first
        temps1t1 = s1t1 #hold the s1 and t1 terms safe for use to generate the new s2t2
        s1t1 = s2t2 #assign s2t2 to s1t1 since last became first
        s2t2 = [temps1t1[0] - q * s2t2[0], temps1t1[1] - q * s2t2[1]] #generate new values for s2t2
    while (s1t1[0] <= 0):
        s1t1[0] = s1t1[0] + phi
    return s1t1

### 2.3
#### Second tool set

Here we will implement the meat of the RSA cryptosystem. The functions below will generate the public and private key pairs which will then be used to create a ciphertext using the public key and then decode the same using the pirvate key.



In [7]:
'''
Finds a public key e for the provided prime numbers p and q.
@param p (int) a prime number
@param q (int), a prime number, not equal to p
@output e (int), the public key, not equal to p or q and pairwise prime to (p-1)(q-1)
'''

def Find_Public_Key_e(p, q):
    phi = (p - 1)* (q - 1) #e must be pairwise prime to (p-1)(q-1)
    e = 2 #starting point for testing potential e values
    while (e <= phi): #loop through values of e, testing each for being pairwise prime with phi
        if (Euclidean_Alg(e, phi) == 1 and e != p and e != q):
            return e
        e +=1 #increment e to test next integer
    return -1 #if e isn't found

In [8]:
'''
Generate private key d to revert encoded message
@param e (int), the public key
@param p (int), the first prime used to generate n
@param q (int), the second prime used to generate n
@output the modular inverse of e mod (p-1)(q-1)
'''

def Find_Private_Key_d(e, p, q):
    phi = (p-1) * (q-1) #generate (p-1)(q-1)
    return bezout_co(e, phi)[0] #use bezout coefficient function to generate modular inverse (first bezout coefficient)

# 3.
#### Putting things all together.

1. In this part, you will define two functions `Encode` and `Decode` which will use the public and private keys that you calculated using the above 2 functions in the second toolset.
2. Using the public key, the `Encode` function will encode a message and generate the corresponding cipher_text.
3. Using the private key, the `Decode` function will decode a ciper_text and recover the original message.



In [9]:
'''
Converts a message into a RSA-encrypted message with public keys e and n
@param n (int), public key modulus, the product of two primes
@param e (int), public key exponent
@param message (string), the message to be encoded
@output a list of integers which each represent one encrypted character from message
'''


def Encode(n, e, message):
    cipher_text = [] #empty list to hold results
    converted_text = Convert_Text(message) #convert text to list of integers for encryption
    for char in converted_text: #for each integer in converted_text, encrypt using FME function with public keys e and n
        cipher_text.append(FME(char, e, n)) #add each encrypted character to output list
    return cipher_text

In [10]:
'''
Converts encoded cipher text (as a list of integers) into readable text
@param n (int), modulus of the public key
@param d (int), private key
@param cipher_text [list], a list of integer values corresponding to encoded characters
@Output message converted to readable, unencrypted characters
'''

def Decode(n, d, cipher_text):
    decoded_list = []
    for char in cipher_text:
        char = FME(char, d, n)
        #char = (char**d) % n
        decoded_list.append(char)
    message = Convert_Num(decoded_list)
    return message

# 3a

### Additional Functions

I am placing additional functions that are necessary for the main function and other functions below here so that they will be executed before they are needed. Please run all of these code blocks before running the main function. Note that the functions necessary to perform some tasks in the main function (such as breaking a public key or manually entering a new value for n) are located below the main function in the notebook and should be run before attempting to access those functions.

Contents:

* Square_Root: a function to compute a square root

* Generate_Prime_List: a function to generate a list of prime numbers large enough for use in breaking or generating new public keys

* Initialize_ned: a function that generates a random public key n and the associated values for public key e, private key d, and factors p and q.

* Print_Menu: a hosuekeeping function that prints various user-selection menus in the main function

In [27]:
'''
This function uses the Sieve of Eratosthenes to generate a list of prime numbers.
The upper limit of the list is tied to an input value n, which will be the largest value
of the public key n used in the main program.
If working with public keys of a similar size, this list can be reused. If a larger public key is
used, the list will be regenerated.
@input n (int), the largest expected value of the public key n
@output a list of prime numbers up to the square root of n
'''
import math

def Generate_Prime_List(n):
    prime_index = [] #create empty list to hold 1000 boolean values at a time (due to memory limits) 
    prime_list = [2] #list to hold the prime numbers generated, filled with 2 to make while loop testable
    prime_list_multiple = [2]
    limit = int(math.sqrt(n)+0.5) #the largest prime we need will be n^(0.5) to conserve complexity
    #print(limit) #For testing purposes
    print("Generating Prime List") #indicates program is working since it may take some time
    counter = 100 #used to print progress reports as primes are generated
    index_start = 0 #These control how large the blocks of prime_index are that are added as the list grows.
    index_end = 5000 #default size is 5000. If changed here, this will carry over to change the list in its entirety.
    index_size = index_end - index_start
    k = prime_list_multiple[0] #initial factor to generate multiples of i
    
    #percent = 0.25 #used to track progress for testing
    
    #Make initial prime_list
    for i in range(index_end): #fill prime_index wil True. True will become false as multiples of prime numbers are "crossed off" according to the sieve
         prime_index.append(True)
    i = 2 #initial prime number
    while ((i * k) < (index_end)): #set all multiples of 2, starting with 4, to false since they are composite  
            prime_index[i*k] = False
            k += 1
            prime_list_multiple[0] = k
    i = 3
    while (i < (index_end)):#iterate through each element of prime_index, looking for True/prime numbers
        if (prime_index[i]): #if boolean at index i indicates i is prime
            k = 2 #reset k for finding multiples of i
            prime_list.append(i) #add i to the prime list
            while ((i * k) < (index_end)): #iterate through every multiple of i and mark composite/False
                prime_index[i*k] = False
                k += 1
            prime_list_multiple.append(k) #store last-used multiple
        i += 2 #increment to next odd number   
    #Iterate until reaching limit
  
    index_start = index_end #move the start and end point of the new list to be checked for primes. Since the prime_index index will always start from 0 to index_size, these are used to generate the actual numbers corresponding to each index
    index_end = index_end + index_size
    while (prime_list[-1] < limit): #stop iterating when prime_list includes sufficient primes to factorize up to n
        for i in range(index_size): #reset prime_index with True up for sqrt(n) values
            prime_index[i] = True
        for i in range(len(prime_list)): #iterate through the prime_index, crossing off multiples of each prime found so far
            prime = prime_list[i]
            k = prime_list_multiple[i] #recall largest multiple used for each prime so far
            while (prime * k < index_end): #loop until multiples exceed limits of this new list section
                prime_index[prime * k - (index_start)] = False #subtracting index_start from the multiple will match the result to the python list index (0 - index_size); "cross off" any multiples found
                k += 1 #move to next multiple
            prime_list_multiple[i] = k    
        i = prime_list[-1] + 2
        while (i <= (index_end)):#iterate through each element of prime_index, looking for True/prime numbers
            if (prime_index[i - index_start]): #if boolean at index i indicates i is prime
                k = 2 #reset k for finding multiples of i
                prime_list.append(i) #add i to the prime list
                while ((i * k) <= (index_end)): #iterate through every multiple of i and mark composite/False
                    prime_index[i*k - index_start] = False
                    k += 1
                prime_list_multiple.append(k) #store last-used multiple
            i += 2 #increment to next odd number   
        index_start = index_end #increment index to next chunk
        index_end = index_end + index_size
        #if (prime_list[-1] / limit > percent): #Used for testing purposes to show progress for large values of n
        #    print(percent_complete, " complete")
        #    percent_complete = str(percent*100) + "%"
        #    percent = percent + 0.25
    print("List Complete")
    return prime_list

In [12]:
'''
This function generates random initial values for n, e, and d
@param prime_list [list], a list of prime numbers used to pull values of p and q
@output a list containing n, e, d, p, and q
'''
import random

def Initialize_ned(prime_list):
    nedpq = [0, 0, 0, 0, 0] #list to hold values of n, e, and d
    p = random.randint(0,len(prime_list)-1) #set initial values for p and q so loop will work
    q = random.randint(0,len(prime_list)-1)
    while (p == q): #set p and q to a random index of prime_list until both are different
        p = random.randint(0,len(prime_list)-1)
        q = random.randint(0,len(prime_list)-1)
    p = prime_list[p] #set p and q to the random indexes on the prime list
    q = prime_list[q]
    nedpq[0] = p * q #generate n
    nedpq[1] = Find_Public_Key_e(p, q) #generate p
    nedpq[2] = Find_Private_Key_d(nedpq[1], p, q) #generate q
    nedpq[3] = p #store p
    nedpq[4] = q #store q
    return nedpq

In [13]:
'''
This function prints the menus in the main function
@input selection (str), string indicating which menu to print
output returns none but prints as part of execution
'''

def Print_Menu(selection):
    if (selection == "initial"): #Main menu
        print("Please enter a selection from the following options:")
        print("e) Encrypt a message")
        print("d) Decrypt a message")
        print("b) Break a private key")
        print("g) Generate/enter new keys")
        print("q) Quit")
    elif (selection =="d"): #Menu for decryption
        print("Choose from the following: ")
        print("m) Decrypt manual cipher entry")
        print("f) Decrypt from file")
        print("p) Decrypt previously generated cipher")
        print("b) Back to main menu")
    elif (selection == "g"): #menu for key generation or input
        print("Choose from the following: ")
        print("r) Random values")
        print("m) Manual 'n' and 'e'")
        print("f) From file")
        print("b) Back to main menu")
    return

### 4. 

### Demonstrate how your RSA works below using a mix of text and code blocks that make it clear to the reader how this works and why you chose to put it together the way you did:

* Encode a message (including generating keys).

* Publish your public key and message to Piazza.

* Decode messages from Piazza.

* Test!





Here I will walk through the process of encoding my piazza prompt and decoding the first response.

#### Encode a Message: 

"What is the opening or closing line of a book you like? Mine: 'The day my wife left, she gave me a list of who I was.' - Native Speaker, Chang-Rae Lee"

In order to encode a message, we must first generate a set of public keys. We do this using the Find_Public_Key_e algorithm, which requires two primes, p and q, as an input. I chose as my two primes p = 101 and q = 229, which give us an n of 23129 (n = p * q). The algorithm chooses e such that e is pairwise prime to (p-1)(q-1) and isn't equal to p or q. This is to ensure that a modular inverse of e mod (p-1)(q-1) exists (so we can "undo" e later with our private key.)

In [14]:
print(Find_Public_Key_e(101, 229))

7


Find_Public_Key_e(101, 229) returns a value of 7 for e.

To encode a message, we must first convert a message in string format to a list of integers because the RSA encoding algorithm must be applied to an integer.

This is done with the Convert_Text function, which takes a string as an input and outputs a list of integers which correspond to the ascii values of the string.

Next, we encrypt the list of integers by performing M^e mod n on each character and creating a new list out of the result. Instead of the standard python modular operator, we use Fast Modular Exponentiation to more efficiently reduce the each character modulo n. The resulting integer values form the cipher.

In the notebook, these programs are combined within the Encode function.

In [15]:
print(Encode(23129, 7, "What is the opening or closing line of a book you like? Mine: 'The day my wife left, she gave me a list of who I was.' - Native Speaker, Chang-Rae Lee"))

[11104, 20670, 17552, 15047, 12967, 16182, 19499, 12967, 15047, 20670, 21513, 12967, 8070, 837, 21513, 12335, 16182, 12335, 7703, 12967, 8070, 16912, 12967, 15628, 6251, 8070, 19499, 16182, 12335, 7703, 12967, 6251, 16182, 12335, 21513, 12967, 8070, 9293, 12967, 17552, 12967, 23063, 8070, 8070, 12488, 12967, 23061, 8070, 3514, 12967, 6251, 16182, 12488, 21513, 17936, 12967, 12407, 16182, 12335, 21513, 17645, 12967, 11656, 15540, 20670, 21513, 12967, 10402, 17552, 23061, 12967, 4634, 23061, 12967, 12168, 16182, 9293, 21513, 12967, 6251, 21513, 9293, 15047, 5284, 12967, 19499, 20670, 21513, 12967, 7703, 17552, 5670, 21513, 12967, 4634, 21513, 12967, 17552, 12967, 6251, 16182, 19499, 15047, 12967, 8070, 9293, 12967, 12168, 20670, 8070, 12967, 1959, 12967, 12168, 17552, 19499, 14955, 11656, 12967, 12089, 12967, 11712, 17552, 15047, 16182, 5670, 21513, 12967, 9850, 837, 21513, 17552, 12488, 21513, 16912, 5284, 12967, 9823, 20670, 17552, 12335, 7703, 12089, 22600, 17552, 21513, 12967, 8911, 

Our encoded text is returned as a list of integers.

Before publishing to piazza to elicit a response, I also had to generate a private key in order to decode the encoded message. This is done with the Generate_Private_Key_d function. d will be the modular inverse of e, so that it will "undo" the encryption when applied to the encrypted characters. To find d, we use the extended euclidean algorithm to find the bezout coefficients of e and (p-1)(q-1). The bezout coefficient of e is the modular inverse of e mod (p-1)(q-1)

In [16]:
print(Find_Private_Key_d(7, 101, 229))

19543


This yields a d of 19543.

#### Decode a message

I published the keys and cipher on piazza to elicit a response. (I had to post the private key as well so that the class could read my message. In real world practice, you would submit a message to someone else using their public keys and they would reply using yours.)

The first response was from Beth, with the end to Wuthering Heights. Using the Decode function, with inputs of n, d, and the cipher, I decoded this message. The Decode function works very similar to the Encode function: each encrypted character is raised to d and then reduced modulo n. Because d is the modular inverse of e, this yields the original character. Finally, the Convert_Num function turns the integers that now represent ascii values back into their respective characters, yielding the decrypted message.

In [17]:
print(Decode(23129, 19543, [11656, 1959, 12967, 6251, 16182, 12335, 7703, 21513, 16912, 21513, 10402, 12967, 16912, 8070, 3514, 12335, 10402, 12967, 15047, 20670, 21513, 4634, 5284, 12967, 3514, 12335, 10402, 21513, 16912, 12967, 15047, 20670, 17552, 15047, 12967, 23063, 21513, 12335, 16182, 7703, 12335, 12967, 19499, 12488, 23061, 17645, 12967, 12168, 17552, 15047, 15628, 20670, 21513, 10402, 12967, 15047, 20670, 21513, 12967, 4634, 8070, 15047, 20670, 19499, 12967, 9293, 6251, 3514, 15047, 15047, 21513, 16912, 16182, 12335, 7703, 12967, 17552, 4634, 8070, 12335, 7703, 12967, 15047, 20670, 21513, 12967, 20670, 21513, 17552, 15047, 20670, 12967, 17552, 12335, 10402, 12967, 20670, 17552, 16912, 21513, 23063, 21513, 6251, 6251, 19499, 5284, 12967, 6251, 16182, 19499, 15047, 21513, 12335, 21513, 10402, 12967, 15047, 8070, 12967, 15047, 20670, 21513, 12967, 19499, 8070, 9293, 15047, 12967, 12168, 16182, 12335, 10402, 12967, 23063, 16912, 21513, 17552, 15047, 20670, 16182, 12335, 7703, 12967, 15047, 20670, 16912, 8070, 3514, 7703, 20670, 12967, 15047, 20670, 21513, 12967, 7703, 16912, 17552, 19499, 19499, 5284, 12967, 17552, 12335, 10402, 12967, 12168, 8070, 12335, 10402, 21513, 16912, 21513, 10402, 12967, 20670, 8070, 12168, 12967, 17552, 12335, 23061, 8070, 12335, 21513, 12967, 15628, 8070, 3514, 6251, 10402, 12967, 21513, 5670, 21513, 16912, 12967, 16182, 4634, 17552, 7703, 16182, 12335, 21513, 12967, 3514, 12335, 16097, 3514, 16182, 21513, 15047, 12967, 19499, 6251, 3514, 4634, 23063, 21513, 16912, 19499, 12967, 9293, 8070, 16912, 12967, 15047, 20670, 21513, 12967, 19499, 6251, 21513, 21513, 837, 21513, 16912, 19499, 12967, 16182, 12335, 12967, 15047, 20670, 17552, 15047, 12967, 16097, 3514, 16182, 21513, 15047, 12967, 21513, 17552, 16912, 15047, 20670, 14955, 11656, 12967]))

'I lingered round them, under that benign sky: watched the moths fluttering among the heath and harebells, listened to the soft wind breathing through the grass, and wondered how anyone could ever imagine unquiet slumbers for the sleepers in that quiet earth.' 


And there we have a return message, decrypted to read: 'I lingered round them, under that benign sky: watched the moths fluttering among the heath and harebells, listened to the soft wind breathing through the grass, and wondered how anyone could ever imagine unquiet slumbers for the sleepers in that quiet earth.' 

#### Respond to a Prompt

Responding to a prompt is the opposite process, in a way. A lot of the values are given with the initial message.

For example, Hitomi Imai posted:

Public key (n, e) = (2173, 1029)

Private key d = 1229

Encoded Message = [336, 703, 1247, 259, 73, 10, 476, 516, 516, 73, 1207, 363, 73, 1497, 1651, 1946, 1549, 73, 991, 363, 1110, 259, 73, 1207, 476, 1966, 73, 176, 1946, 1549, 261, 703, 1247, 979, 363, 47, 73, 1303, 73, 1247, 260, 73, 259, 703, 476, 991, 1909, 476, 991, 1966, 73, 259, 1651, 73, 1207, 1946, 1497, 73, 1247, 73, 991, 363, 10, 73, 1549, 363, 1765, 1549, 476, 1966, 363, 1549, 1247, 259, 1651, 1549, 49]

To understand/decode the prompt, I plug n and d into my Decode function. e isn't necessary to read the message, but it will be necessary to generate a new one.

In [32]:
print(Decode(2173, 1229, [336, 703, 1247, 259, 73, 10, 476, 516, 516, 73, 1207, 363, 73, 1497, 1651, 1946, 1549, 73, 991, 363, 1110, 259, 73, 1207, 476, 1966, 73, 176, 1946, 1549, 261, 703, 1247, 979, 363, 47, 73, 1303, 73, 1247, 260, 73, 259, 703, 476, 991, 1909, 476, 991, 1966, 73, 259, 1651, 73, 1207, 1946, 1497, 73, 1247, 73, 991, 363, 10, 73, 1549, 363, 1765, 1549, 476, 1966, 363, 1549, 1247, 259, 1651, 1549, 49]))

What will be your next big purchase? I am thinking to buy a new refrigerator.


To return a message, I use the Encode function, now with Hitomi's given values for n and e:

In [ ]:
print(Encode(2173, 1029, "We bought a new fridge this year. Lifechanging! haha. I think next is a treadmill for winter.")

I can check if this worked correctly by running it back through Decode. If successful, it will print a recognizable string. If not, it will display gibberish.

If you create a custom main function or a script to read codes, you may include it here:

In [18]:
'''
Reads a text file containing a public key modulus (n), public key exponent (e), private key (d), and coded message (separated and not contained within brackets)
text file should be contained within project folder and be formatted as follows (separated by lines):

n
e
d
code

@param none
@output list containing [n, e, d, encrypted message]
'''

def Decode_File():
    codefile = input("Enter the filename: ") #user inputs filename
    codefile = open(codefile, 'r') #open file
    file_content = codefile.readlines() #read lines
    for i in range(3): #convert n, e, and d to integers
        file_content[i].strip()
        file_content[i] = int(file_content[i])
    code_block = [] #prepare list for encrypted message
    for number in file_content[3].split(', '): #split code string into separate strings and iterate through each
        code_block.append(int(number)) #convert each 4-digit code entry to an integer and add to code_block
    file_content[3] = code_block #replace code string with code integer list
    return file_content #return decoded string

In [33]:
'''This block contains the main function'''


def main():
    print("This program will encrypt and decrypt messages using the RSA algorithm.")
    print("Before we begin, I will generate a new public and private key. You will have the opportunity to select your own later.")
    largest_n = 10000 #record largest number used to generate prime_list. If breaking a public key larger than n, a new list will be necessary
    prime_list = Generate_Prime_List(largest_n) #initialize prime_list
    nedpq = Initialize_ned(prime_list) #initialize n, e, and d
    cipher = [] #initialize cipher outside of while loop
    print("Initial values: Public key n = {}, public key e = {}, private key d = {}".format(nedpq[0],nedpq[1],nedpq[2]))

    selection = "initial"
    Print_Menu(selection) #display initial menu
    selection = input()
    while (selection != "q"): #test for quit input

        if (selection == "e"): #Encryption section              
            message = input("Enter the message to encrypt:") #user enters message to encrypt
            print("Encrypted message: ")
            cipher = Encode(nedpq[0], nedpq[1], message) #encrypt message using current values of ned and assign to cipher
            print(cipher) #print cipher
            print("")

        elif (selection == "d"): #Decryption section
            Print_Menu(selection) #display decryption menu
            selection = input() #select decryption option
            while (selection != "b"): #test for valid input
                if (selection == "m"): #decrypt a user-input message
                    cipher = input("Enter the cipher to decrypt (no brackets): ")
                    cipher = cipher.split(", ") #convert string input into list of integers and reassign to cipher
                    for i in range(len(cipher)):
                        cipher[i] = int(cipher[i])
                    break
                elif (selection == "f"): #decrypt from a text file in same folder
                    file_content = Decode_File() #assign contents of file to file_content
                    nedpq[0:3] = file_content[0:3] #update ned from file values
                    cipher = file_content[3] #update cipher from file
                    break
                elif (selection == "p"): #do not change cipher (used to test encryption function)
                    break
                selection = input()
            print("")
            print("Decrypted message: ")
            print(Decode(nedpq[0], nedpq[2], cipher)) #print decrypted cipher using current values of ned and cipher
            print("")

        elif (selection == 'b'): #Code breaking section
            nedpq[0] = int(input("Enter the public key n that you wish to crack: "))
            nedpq[1] = int(input("Enter the public key e that you wish to crack: "))
            nedpq[2] = Force_d_PR(nedpq[0], nedpq[1]) #crack d and assign to d
            print("New values: n = {}, e = {}, d = {}".format(nedpq[0], nedpq[1], nedpq[2]))
            print("")

        elif (selection == 'g'): #Updating n, e, and d section
            Print_Menu(selection)
            selection = input()
            while (selection != "b"): #test for valid input
                if (selection == "r"): #update randomly
                    min_n = 10**(int(input("Enter the minimum number of digits for n: "))-1)   
                    if (min_n >= largest_n): #test if prime list can support minimum digits
                        largest_n = min_n * 10
                        prime_list = Generate_Prime_List(largest_n) #generate a new prime list
                    nedpq = Initialize_ned(prime_list) #generate new ned
                    counter = 0
                    while (nedpq[0] < min_n): #loop until values found for ned that exceed minimum value
                        nedpq = Initialize_ned(prime_list)
                        counter += 1
                        if (counter > 10000):
                            print("Process Failed")
                            break
                    break
                elif (selection == "m"): #update manually
                    nedpq[0] = int(input("Please enter new value for n: ")) #user enters new n
                    nedpq[3] = Pollard_Rho(nedpq[0]) #factorize new p from new n
                    nedpq[4] = int(nedpq[0] / nedpq[3]) #use new p to determine new q
                    print("Would you like to update e automatically?") #ask user if they want to enter a specific e or generate a new one automatically
                    print("y) Yes, update for me.")
                    print("n) No, update manually.")
                    selection = input()
                    good_input = False #control for input that isn't "y" or "n"
                    while (not good_input):
                        if (selection == "y"): #use Find_Public_Key_e and Find_Private_Key_d to generate e and d for new n
                            good_input = True
                            nedpq[1] = Find_Public_Key_e(nedpq[3], nedpq[4])
                            nedpq[2] = Find_Private_Key_d(nedpq[1], nedpq[3], nedpq[4])
                        elif (selection == "n"): #allow user to input new value for e
                            good_input = True
                            nedpq[1] = int(input("Please enter a new value for e: "))
                            nedpq[2] = Find_Private_Key_d(nedpq[1], nedpq[3], nedpq[4]) #generate new value for d based on e and n
                        else:
                            selection = input()
                    break
                elif (selection == "f"): #read in ned from file
                    file_content = Decode_File() #assign contents of file to file_content
                    nedpq[0:3] = file_content[0:3] #update ned from file values
                    break
                selection = input()

            print("New Values: Public key n = {}, public key e = {}, private key d = {}".format(nedpq[0],nedpq[1],nedpq[2]))
            print("")

        selection = "initial"
        Print_Menu(selection) #display initial menu again
        selection = input()
    print("Quitting Program")
                    
if __name__ == '__main__':
    main()

This program will encrypt and decrypt messages using the RSA algorithm.
Before we begin, I will generate a new public and private key. You will have the opportunity to select your own later.
Generating Prime List
List Complete
Initial values: Public key n = 1137371, public key e = 7, private key d = 324343
Please enter a selection from the following options:
e) Encrypt a message
d) Decrypt a message
b) Break a private key
g) Generate/enter new keys
q) Quit


 q


Quitting Program


### 5.


**Insert Narrative (one or 2 pages) here in this block describing the results of exchanging keys and codes with classmates.**

This was a very challenging and rewarding project!

The initial challenge was simply in understanding RSA. The biggest leap here for me came from seeing the math done on a small scale. Ben kim posted this video: https://www.youtube.com/watch?v=Z8M2BTscoD4 which was very helpful for understanding RSA. An anonymous poster in piazza @131 made an analogy to the public keys being more akin to an "envelope" than a message was also incredibly helpful. The official class videos on RSA and all of the individual algorithms (EEA, FME, binary conversion) were also invaluable, as were the MIT lecture notes (which Kevin Vick attested to, convincing me to check them out more fully). I'm glad I had made a version of Square and Mod for the problem set, because it helped not having a full algorithm to follow in order to help me understand what is happening in Siriam's FME algorithm.

I think the most challenging of the basic algorithms was the one used to compute bezout coefficients. I spent a lot of time trying to make sure the two inputs for the gcd(a,b) portion were in order (larger, smaller) as this was emphasized in the lectures. As a result of my tortured attempts to keep everything straight, I often ended up with the coefficients being reversed at the end. Here's where I realized the value of *understanding* something rather than simply applying it, as for a computer it does not matter which order the inputs are in! The first round of EEA simply reverses the two inputs without changing them if they are out of order. The tip to keep them in order was meant to help us perform the calculations by hand. By deciding to do a few test cases, I realized this and the algorithm was much easier to build.

I will say, one of the more frustrating parts of this particular project is that n is used as a key term in RSA (the public key) and is also used as a standard "input" variable in python (for example in the FME algorithm demonstrated in the lectures.) Looking back over the code, it can be difficult to discern if a certain n is n, the public key, or just n, a generic stand-in. In the future, I will probably rely on more descriptive terms in my functions to avoid this confusion!

Some other mistakes and what I learned from each:

* Forgetting to copy n, e, or d into the function when encoding or decoding messages on Piazza. This was a simple mistake with a simple correction, however seeing how each of those pieces (n, e, and d) had to be correct for the message to come out in plaintext at the end underscored how each one was uniquely built in relation to the others. This helped solidify the prerequisites of each (i.e. e is pairwise prime to (p-1)(q-1) or d is modular inverse to e.)

* I attempted to make a square root function that relied on Newton's Method because I thought it might be similar to the native python modulus operator and FME, where there was a more efficient way for large numbers particularly when I only needed the units level of precision. The method worked, but seemed to take slightly longer than math.sqrt, so I abandoned it. I would like to find a source for how the simple python functions work to determine if there is room for building more efficiency or not.

* A mistake that helped me a lot with my understanding of the project was anothers: another student had posted on piazza with an encoded message and her keys. I was able to decode it as with other messages, but when I tried to encode a response, it came out as gibberish when I tested it by decoding the response again. At first, I assumed this was my mistake, but other messages still worked fine. Then, I wanted to let her know, but without posting something in the open. This lead me to try to figure out how to reverse engineer the e public key. I already had a function that would give me d. I thought I could use d to determine the modular inverse, but realized that this required knowledge of (p-1)(q-1). Using my factoring function instead of my Force_d function, I could identify the p and then use n / p to find q, then use p and q to generate a new e based on the given n. It wasn't until I went through this process that I really understood how all the different pieces together. Later on, when I built my main function, I used this experience to help figure out how to automatically generate an e, given a user-provided n. (It also caused me to rewrite the list that contained n, e, and d to also include p and q! It is useful to know p and q and hard to recover them!)

A general software development lesson learned was that building a script to read codes from text files or a basic main function should be done early! It is so much easier to test everything if you don't have to copy and paste keys from one code block to another and between two open internet browsers.

The most intriguing result came from the code breaking thread, where Mariana Lenetis posted the following code: [100000, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 169168, 63125, 63125, 63125, 169168, 63125, 63125, 169168, 100000, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 109165, 63125, 63125, 109165, 63125, 63125, 109165, 100000, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 169168, 63125, 63125, 169168, 63125, 63125, 169168, 100000, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 27067, 207173, 207173, 207173, 207173, 207173, 207173, 207173, 27067, 100000, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 137647, 207173, 207173, 207173, 207173, 207173, 207173, 207173, 201466, 187816, 27067, 100000, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 193230, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 193230, 63125, 193230, 100000, 63125, 63125, 63125, 63125, 180166, 5805, 98075, 32000, 178644, 190597, 5805, 63125, 91601, 178644, 63125, 207204, 23042, 26626, 63125, 32000, 178644, 113300, 5805, 63125, 117983, 207204, 5805, 26626, 23011, 14861, 56819, 235508, 72486, 63125, 63125, 63125, 193230, 63125, 63125, 63125, 63125, 63125, 63125, 63125, 193230, 187816, 201466, 100000, 63125, 63125, 63125, 63125, 149017, 178644, 25676, 98075, 113300, 63125, 188782, 178644, 25676, 63125, 98075, 14861, 23011, 5805, 63125, 32000, 178644, 78018, 78018, 5805, 5805, 63125, 178644, 207204, 63125, 91601, 5805, 26626, 48737, 63125, 63125, 63125, 63125, 137647, 207173, 207173, 207173, 207173, 207173, 201466, 100000, 63125, 63125, 63125, 63125]

Which decrypts to an ascii picture of a steaming coffee cup with a message (that doesn't translate well to this markdown box because of all the special characters...) 

This is a puzzle to me. I tried copying and pasting this back into my Encode function, but the spacing did not translate. I will need to build a function that encrypts from a text file, I think, to perhaps be able to recreate her results. I also notice two different characters in teh code above: "100000" and "63125" that both seem to indicate a blank space. The former seems to be the newline character. This definitely has aroused my curiousity and, if I can make the codebreaking work efficiently, will be my next investigation.

### 6.

FME Exploration

Pick two examples of decoding or encoding a message that use **a very large n and a very small n**. This could be a code you used or a new one. Time how long it takes to calculate each (look up Python Timing function, or informally mark the time). Then comment out your FME code and just use the Python Mod function in your FME function. Is the difference significant? Discuss.



I experimented using the timit() function on two different n values.

For n = 5251: (5 repetitions)

* FME took 0.0036 seconds

* % took 0.042 seconds

For n = 50250256062319221: (1 repetitions)

* FME took 0.003 seconds

* % took in excess of an hour and still did not finish.

This shows that even with modest values of n, the default python modular operator is extremely inefficient compared to FME. Since performing modular operations is part of producing a public and private key, this would render the creation of strong keys impossible without FME or a similar efficient algorithm.

## CODE BREAKING ##

This section of the project is only available to students exchanging codes by the Friday (midnight) before the project is due.

After you have tested your RSA package yourself, and tested it with classmates by publishing both the private and public keys on Piazza, post 2 messages with just the public keys for people to break on the "Just Public Keys" thread - use both very small n ‘s (under 1000) for practice, and some with a challenge.


**Implementing a factoring algorithm:**

Begin by coding the basic brute force factorization algorithm given in pseudocode below.

Brute Force Factoring
<pre><code>def factorize(n):
    # n is a number, return the smallest factor of n
    for i from 2 to n-1:
        if i divides n:
            return i
        return FALSE
        </code></pre>


In [21]:
'''
Basic function to factor a given integer using FME.
@param n (int), the number to factorize
@output the least factor found between 2 and n-1
'''
def Factorize_Basic(n):
    # n is a number, return the smallest factor of n
    for i in range(2,n-1): #iterate from 2 to n-1
        if FME(n,1,i) == 0: #determine if i divides n
            return i #if so, return i as a factor
    return False


### 7.
* To break others’ code you just need to factor n (public key). Explain why and how this works in a text block.

* Demonstrate 1 example of breaking a code with a small n, and show the steps involved.

* Now attempt codes with larger n

* Describe your attempt to break codes with large n. At what values would you say n's get difficult? (too large to factor)





#### Breaking Codes

To generate either key (the public e or the private d), you need to know p and q, the prime factors that make up n. Even if e is unavailable, by factoring n, you can determine p. By dividing n by p, you determine q. With both p and q, you can determine e. Finally, with e, p, and q, you can determine d.

To decode, you need only n and d.

To break a code:

Example: n = 671

* Factor n: using the Factor_Basic method, we get p = 11
* Divide n by p to get q: 671 / 11 = 61
* Find e: with Find_Public_Key_e, we get e = 7
* Use e, p, and q to find d: with Find_Private_Key_d, we get d = 343

On piazza, this public key was posted by Dylan Power with the following cipher: [617, 133, 515, 515, 326, 326, 395, 133, 577, 395, 481, 326, 598, 189]

Decrypted, it reads: "Coffee or tea?"

In [22]:
print(Factorize_Basic(671))
print(Find_Public_Key_e(61, 11))
print(Find_Private_Key_d(7, 61, 11))
Decode(671, 343, [617, 133, 515, 515, 326, 326, 395, 133, 577, 395, 481, 326, 598, 189])

11
7
343


'Coffee or tea?'

The basic method is not effective for larger values of n.

I have combined the steps above into a function Force_d_Basic to determine d.

For n = 1984909, the basic factoring worked practically instantly.
For n = 502560652319221, it took significantly longer (about 20 seconds).
For n = 28794290241716387, it took 4-5 minutes.

In [23]:
'''
This function uses the basic Factorize_Basic function to determine private key d from public keys n and e.
@input n (int) public key n
@input e (int) public key e
@output private key d
'''

def Force_d_Basic(n, e):
    p = Factorize_Basic(n) #find a prime factor of p using inefficient algorithm
    q = int(n / p) #divide n by p to find the other prime factor q
    d = int(Find_Private_Key_d(e, p, q)) #with all inputs available, find d using e, p, and q
    return d

print(Decode(502560652319221,Force_d_Basic(502560652319221, 5),[1350125107, 16850581551, 16105100000, 11592740743, 19254145824, 8587340257, 21003416576, 20113571875, 33554432, 16850581551, 16105100000, 33554432, 9509900499, 19254145824, 8587340257, 9509900499, 14025517307, 12762815625, 16105100000, 11592740743, 33554432, 21003416576, 12166529024, 10510100501, 33554432, 9509900499, 16850581551, 10000000000, 10510100501, 33554432, 23863536599, 12762815625, 21003416576, 12166529024, 33554432, 8587340257, 33554432, 2887174368, 33554432, 12762815625, 16105100000, 33554432, 21003416576, 12166529024, 10510100501, 33554432, 21003416576, 19254145824, 12762815625, 14693280768, 14693280768, 12762815625, 16850581551, 16105100000, 20113571875, 39135393, 39135393, 39135393, 39135393]))

Congrats on cracking the code with a N in the trillions!!!!


### 8. 

Now implement at least **two** improvements to your factoring method. (Should be 2 code blocks - include useful comments)

To improve the brute force method - a very naive and slow approach in many ways - we recommend considering improvements in the following order:

**Basic Improvement #1 :** Skip over certain numbers (pick at least one of these):

The brute force approach above tests even numbers as well as odd. If the number is odd, there is no point searching for an even factor.

We can do a better job tackling multiples of 3. If the number is not a multiple of 3, then no factor can be a multiple of 3 either.

How about multiples of 5,7,11,13,…?

**Basic Improvement #2: The brute force approach iterates all the way to n−1. Can we stop earlier?**


Hint: Explain with examples how using THM 2 page 258 could improve the brute force method (and/or other methods) of factoring.

Then add this technique to your factoring code to improve it.



See advanced options in the project description in Moodle.


 



I have implemented two improvements:

First, I use a function Generate_Prime_List(n) (included in part 3a of this notebook) which generates all prime numbers up to and including the square root of n using the Sieve of Eratosthenes. Then, in Factorize_Improved, I test only the prime numbers on that list for divisibility. This reduced the time to factor by skipping numbers that will not be factors (since they themselves have prime factors) and stopping at square root n (before which, there must be a prime factor if the number is composite according to THM 2.)

The generation of the prime list is the bottleneck here. If n is too large, it can take longer to generate the list than it will to use the naive factoring algorithm. The advantage of the list is that, once generated, it can be reused for any n smaller than the initial input. Were I to build it again, I think I would integrate the process so that as the prime list was built, it tested n for factors every 5000 primes or so. This would cost little extra time and prevent every case from being the worst case (must generate all primes before factoring) as it is now.

Unfortunately with the limits in JupyterHub, I could not break the code below online. I broke it on my computer instead.

----

I will now break a larger code: n = 28794290241716387, e = 113

I have combined the steps above into a function Force_d to determine d.

Force_d yields d = 23697955405914833

The cipher reads: [2579706886152660, 1506880588871094, 25824199538612172, 12324506914914625, 3910629358746840, 19942730250851363, 12407119628852595, 20436916271957116, 22358653510094960, 22358653510094960, 18133059900110149, 19942730250851363, 18350208408879280, 12324506914914625, 27102085920647667, 28009467377098228, 19348565858364802, 19942730250851363, 8479118201845191, 28009467377098228, 27102085920647667, 19348565858364802, 8302426957980714, 19270259444554612, 19942730250851363, 8302426957980714, 13211641791567831, 25824199538612172, 19942730250851363, 8097851867855643, 20436916271957116, 3910629358746840, 18133059900110149]

Decoded, it reads: Every Fall, Orion points the way

Here, I will demonstrate with the slightly smaller n of 502560652319221. Where before, with the basic method, factoring took about 20 seconds. Now, once the prime list is generated, it takes labout 1-2 seconds.

Code follows:

In [ ]:
demo_prime_list = Generate_Prime_List(502560652319221)


Generating Prime List


In [24]:
print(Decode(502560652319221, Force_d(502560652319221, 5, demo_prime_list), [1350125107, 16850581551, 16105100000, 11592740743, 19254145824, 8587340257, 21003416576, 20113571875, 33554432, 16850581551, 16105100000, 33554432, 9509900499, 19254145824, 8587340257, 9509900499, 14025517307, 12762815625, 16105100000, 11592740743, 33554432, 21003416576, 12166529024, 10510100501, 33554432, 9509900499, 16850581551, 10000000000, 10510100501, 33554432, 23863536599, 12762815625, 21003416576, 12166529024, 33554432, 8587340257, 33554432, 2887174368, 33554432, 12762815625, 16105100000, 33554432, 21003416576, 12166529024, 10510100501, 33554432, 21003416576, 19254145824, 12762815625, 14693280768, 14693280768, 12762815625, 16850581551, 16105100000, 20113571875, 39135393, 39135393, 39135393, 39135393])) 

Congrats on cracking the code with a N in the trillions!!!!


In [24]:

'''
Improved function that tests only primes up to square root of n as factors of n
@input n (int), public key n
@input prime_list [list], a list of primes up to square root n, generated externally
@output prime factor of n
'''

def Factorize_Improved(n, prime_list):
     # n is a number, return the smallest factor of n
    for prime in prime_list: #only test prime numbers up to the square root of n
        if FME(n,1,prime) == 0: #determine if i divides n
            return prime #if so, return i as a factor
    return False

In [25]:
'''
This function uses the Factorize_Improved function to determine private key d from public keys n and e.
@input n (int) public key n
@input e (int) public key e
@input prime_list [list] list of prime numbers up to square root n, generated by Generate_Prime_List
@output private key d
'''

def Force_d(n, e, prime_list):
    p = Factorize_Improved(n, prime_list) #determine p by factorizing n with the elements of prime_list
    q = int(n / p) #determine q by dividing n by p
    d = int(Find_Private_Key_d(e, p, q)) #find private key d using e, p, and q as inputs
    return d

#### Other Broken Codes:

* Sender: Vu Dang

n: 102186307

e: 3

d: 68091867

Message: "From each according to their abilities, to each according to their needs."

* Sender: Micah JASI9001

n: 20446247615609863829

e:  1799

d: 16434282400830991799

Message: "Ground control to Major Tommm...."


## Custom Feature ##
The last 5 points require a custom feature in your project. This is an opportinity for you to explore an RSA or coding topic of interest to you. At this point it is not about points, but exploring advanced ideas relevant to you.  This is also a feature that should be relevant to your programming ability. If you are in 1300, creating a main type function may be enough. If you are an experienced programmer, find a topic to push yourself. 


Custom Features could be:
1. A Python "main" function that allows a user to "Get keys/Encode/Decode/Break Codes" etc...
2. A script to read codes from Piazza data. You can copy and paste codes from Piazza into a txt files and then have your script read it. This is a VERY helpful tool for doing the project as well.
3. Exploration of advanced factoring alogrithms (see Moodle) Try Pollard's Rho <- super satisfying. 
4. Advanced complexity analysis. 




### 10. Conclusion ##
Your advanced custom feature may be incoporated into the project or as an add-on here.  In either case, use a text block here to descibe why you chose this custom feature(s) and how it fits into the project. 


#### Script function:

I built a simple function that would read a text file placed in the same folder as this notebook. It helped immensely with staying organized while encrypting and decrypting codes posted on piazza (and testing my own.) I added it to the main function as an option for decryption so users can avoid typing in n, e, and d manually.

#### Main function:
I wish I had built this much earlier, as it makes testing and performing all functions of this program much more user-friendly. I implemented some basic input-controls but could improve on those and would probably split out each section (encryption, decryption, etc.) into their own separate modules for neatness if I wanted to improve it. I do like that on startup it initializes a new n, e, and d so users can jump into encrypting a message or soliciting one with only a couple key presses.

One improvement I would make in the future would be to change the Generate_Prime_List function so that if a larger n was given, it would start from where the existing list of primes ended and add on until it had sufficient elements. As it is now, when it needs to grow it starts back from 2 again. I would also try to implement hyperthreading into that algorithm as it is the most resource-intensive portion of the entire project.



#### Pollard's Rho algorithm:

CREDIT to Beth's tutorial and to a tutorial on sofosband (https://sofosband.wixsite.com/pversusnp/single-post/2018/08/27/Factorising-part-2---Pollards-Rho-algorithm)

I attempted, and seem to have succeeded in creating a function that performs Pollard's Rho algorithm to more efficiently factorize n. I used the guides listed above to an embarrassing amount. I first attempted the algorithm using randomly generated numbers for a, b, and c and reset each when the algorithm stalled out. This sometimes succeeded very quickly, and sometimes ran interminably. I then essentially noticed the example algorithms started from the same, simple number (1 or 2) and modified my code to match. That worked much better.

I did restructure to control for:
* resetting the value of a when a and b caught up to each other
* testing for the factor of n, which tended to be the returned result the majority of the time

I chose this because I wanted to see a better method of factoring than my prime-listing method (which took too long with n's of greater than 15 digits... though I believe it would be quite powerful if the prime_list was generated ahead of time in a dedicated code-breaking machine.) I have now built this into the main function as the primary factoring function.

Side note: Using Pollard's Rho gives off a powerful high.

In [26]:
'''
This function generates pseudorandom numbers for the Pollard_Rho algorithm
@input x (int), any integer; normally the previous result of the algorithm
@input c (int), any integer to assist in pseudorandomizing
@input n (int), the number that the Pollard's Rho algorithm is attempting to factorize
@output a pseudorandom number generated by the function x^2 + c mod n
'''

def PR_Function(x, c, n):
    return FME(x * x + c, 1, n) #FME used for efficiency

'''
Pollard's Rho algorithm for factoring. Uses pseudorandomly-generated numbers
to look for a difference that will yield a common factor with n
@input n (int), the number to be factorized
@output one of the factors of n
'''

def Pollard_Rho(n):
    p = 1 #set initial values of p, a, and b so they can be entered into PR_Function
    a = 1
    b = 1
    while (p == 1): #if p != 1, then we have found a factor (the GCD)
        a = PR_Function(a, 1, n) #use PR_Function on the earlier value of a to generate a pseudorandom number
        b = PR_Function(PR_Function(b, 1, n), 1, n) #use the PR_Function twice on the earlier value of b to generate a pseudorandom number
        p = Euclidean_Alg(abs(a-b), n) #Check if a - b has a common factor with n (the GCD of a-b and n), if so, the while loop will exit and return that factor
        if (a == b): #Check if the a (tortoise) and b (hare) loops have caught each other. If so, reinitialize a to a random number
            a = random.randint(0, n)
    return p

'''
This function uses the Pollard_Rho function to determine private key d from public keys n and e.
@input n (int) public key n
@input e (int) public key e
@output private key d
'''

def Force_d_PR(n, e):
    p = Pollard_Rho(n) #determine p by factorizing n with the elements of prime_list
    q = int(n / p) #determine q by dividing n by p
    d = int(Find_Private_Key_d(e, p, q)) #find private key d using e, p, and q as inputs
    return d

## Grading Notes ##

*The project will consist of a combination of text and code blocks.

*Grading - all code must be well commented. These should be easy to read. Here is a simple guide: https://developers.google.com/tech-writing/two/code-comments
 
*The narrative elements of the project are just as important as the coding.

*Do NOT copy code from the internet (or anywhere)

*Do not ask questions on stackoverflow or similar etc…

*You are expected to have read the honesty notes in the syllabus.

*Credit sources you used for additional understanding.

*Credit classmates that helped you.



Other Credits:

To help develop my functions for generating prime number lists: https://www.geeksforgeeks.org/sieve-of-eratosthenes/

Beth's comments on the main function conventions and the RSA pro tips, in addition to the class content, were invaluable.

Kevin's note on JupyterHub symbology and kernel restarting also came in handy for hung functions and restarts.

